In [4]:
import re

In [5]:
def parse_frams_geno(geno:str, in_paranthesis=False):
    if len(geno) == 0:
        return ["empty"]
    j = geno.find(",")
    if j != -1 and in_paranthesis:
        return ["comma"] + parse_frams_geno(geno[:j]) + parse_frams_geno(geno[j+1:],True)
    i = geno.find("(")
    if i != -1:
        l = geno.rfind(")")
        return ["add_branch"] + parse_frams_geno(geno[:i]) + parse_frams_geno(geno[i+1:l],True)

    if len(geno) == 1:
        return [geno[0]]

    if geno[1] == "[": #handle
        match = re.search(r'\[[^\[\]]*?\](?!\[)', geno)
        type, rest = parse_neuron(match.group()[1:-1])
        return ["add_neuron"] + parse_frams_geno(geno[:match.start()]) + [type] + rest + parse_frams_geno(geno[match.end():])
    return ["concat", geno[0]] + parse_frams_geno(geno[1:])

def parse_neuron(neuron_inside:str | list[str], is_first=True):
    if len(neuron_inside) == 0:
        return ["end"]
    if isinstance(neuron_inside,str):
        neuron_inside = neuron_inside.split(",")
    neuron_type = None
    i = 0
    if ":" not in neuron_inside[0]:
        neuron_type = neuron_inside[0]
        i = 1
    if is_first:
        neuron_type = "N" if not neuron_type else neuron_type
        return neuron_type, parse_neuron(neuron_inside[i:], is_first=False)
    
    first, second = neuron_inside[0].split(":")
    if re.match(r'^-?\d+$',first):
        thing = ["conn", int(first), float(second)]
    else:
        thing = ["prop", first, float(second)]

    return thing + parse_neuron(neuron_inside[1:], is_first=False)


In [6]:
def simple_parser(geno:str, in_paranthesis=False):
    if len(geno) == 0:
        return ["end"]
    if in_paranthesis:
        depth = 0
        j = -1
        for i, c in enumerate(geno):
            if c == "(" or c == "[":
                depth += 1
            if c == ")" or c == "]":
                depth -= 1
            if c == "," and depth == 0:
                j = i
                break
        if j != -1:
            return ["comma"] + simple_parser(geno[:j]) + simple_parser(geno[j+1:],True)
    if geno[0] == "(":
        return ["branch"] + simple_parser(geno[1:-1],True)

    if geno[0] == "[": #handle
        #match = re.search(r'\[[^\[\]]*?\](?!\[)', geno)
        i = geno.find("]")
        type, proplist = parse_neuron(geno[1:i])
        return ["neuron"] + [type] + proplist + simple_parser(geno[i+1:])
    return [geno[0]] + simple_parser(geno[1:])

In [7]:
parse_neuron("N,3:2,fi:2")

('N', ['conn', 3, 2.0, 'prop', 'fi', 2.0, 'end'])

In [56]:
simple_parser("(X(X,,,,(X,X,XX(X))))")

['branch',
 'X',
 'branch',
 'comma',
 'X',
 'end',
 'comma',
 'end',
 'comma',
 'end',
 'comma',
 'end',
 'branch',
 'comma',
 'X',
 'end',
 'comma',
 'X',
 'end',
 'X',
 'X',
 'branch',
 'X',
 'end']

In [9]:
%matplotlib inline
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
import networkx as nx

nodes, edges, labels = gp.graph(tree)
g = nx.Graph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
pos = graphviz_layout(g, prog="dot")

nx.draw_networkx_nodes(g, pos)
nx.draw_networkx_edges(g, pos)
nx.draw_networkx_labels(g, pos, labels)
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [11]:
#geno = "cccRRRXXX[N][N]X(X,,X(X,,cXCX,R(RRX)),XX)"
geno = "(cccXX[0:2]XX[*][Thr,lo:2](X,,,,(X,X,XX(X))))"
parsed = simple_parser(geno)
print(*parsed)

branch c c c X X neuron N conn 0 2.0 end X X neuron * end neuron Thr prop lo 2.0 end branch comma X end comma end comma end comma end branch comma X end comma X end X X branch X end


In [ ]:
from deap import gp
from functools import partial

In [16]:
from framspy.src.FramsticksLib import FramsticksLib
from pathlib import Path

In [17]:
flib = FramsticksLib("./Framsticks52",None,Path("./framspy/eval-allcriteria.sim").absolute().as_posix())

Using Framsticks version: 5.2
Home (writable) dir     : d:\studia_2\F1-GP-GOMEA\Framsticks52\data
Resources dir           : d:\studia_2\F1-GP-GOMEA\Framsticks52\data

Available objects: ['CheckpointEvent', 'Collision', 'CrCollision', 'Creature', 'CreatureSettings', 'CreatureSignals', 'CreatureSnapshot', 'Dictionary', 'ExpProperties', 'ExpState', 'ExtValue', 'File', 'FunctionReference', 'GenMan', 'GenManStats', 'GenePool', 'GenePools', 'Geno', 'GenoConverters', 'Genotype', 'Interface', 'Joint', 'Loader', 'Math', 'MechJoint', 'MechPart', 'MessageCatcher', 'Model', 'ModelGeometry', 'ModelSymmetry', 'Neuro', 'NeuroClass', 'NeuroClassLibrary', 'NeuroDef', 'NeuroSignals', 'NeuronsSimEnabled', 'ODE', 'Orient', 'Part', 'Population', 'Populations', 'Ref', 'Signal', 'SignalView', 'SimilMeasure', 'SimilMeasureDistribution', 'SimilMeasureGreedy', 'SimilMeasureHungarian', 'Simulator', 'SlaveSimulators', 'StopEvent', 'String', 'UserScripts', 'Vector', 'WireframeAppearance', 'World', 'WorldMap', 'Wor

In [18]:
def generate_random(n=100, geno_format="1"):
    geno = [flib.getSimplest(geno_format)]
    for i in range(n):
        geno = flib.mutate(geno)
    return geno

In [19]:
N = 30
pop = [generate_random() for i in range(N)] 

In [20]:
class NodeMock():
    def __init__(self, name):
        if isinstance(name,int):
            name = "nint"
        if isinstance(name,float):
            name = "nfloat"
        self.name = name

In [21]:
parsed = [list(map(lambda s: NodeMock(s),simple_parser(geno[0].replace(" ","")))) for geno in pop]

In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from src.linkage import LinkageTreeFramsF1
lt = LinkageTreeFramsF1(parsed,None)

In [45]:
import numpy as np
import itertools
pad_token = "_"
mapped = [list(map(lambda s: s.name, ind)) for ind in parsed]
padded = list(zip(*itertools.zip_longest(*mapped, fillvalue=pad_token)))
genos_array = np.array(padded)

In [174]:
from collections import Counter

In [181]:
single_counts = {i: Counter(genos_array[:, i][genos_array[:, i]!=pad_token]) for i in range(genos_array.shape[1])}

In [183]:
pair_counts = {
            (i, j): Counter(zip(genos_array[:, i][genos_array[:, i]!=pad_token], genos_array[:, j][genos_array[:, j]!=pad_token]))
            for i in range(genos_array.shape[1]) for j in range(i + 1, genos_array.shape[1])
        }

In [185]:
lt.positional_entropies

{0: np.float64(3.360942544894305)}

In [187]:
from src.stats import spearman_cor, pearson_cor, calculate_entropy_list

In [ ]:
pop = []

In [ ]:
def evaluate(ptree, pset, flib, invalid_fitness, criteria):
    try:
        geno = [gp.compile(ptree, pset)]
    except:
        return (invalid_fitness,)
    valid = flib.isValidCreature(geno)[0]
    if not valid:
        return (invalid_fitness,)
    # before running a creature through a simulation we ensure the genotype is valid
    value = flib.evaluate(geno)[0]["evaluations"][''][criteria]
    return (value,) 
